### Check avaiavailable GPU and import MovrNet

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import math

gpus = tf.config.list_physical_devices('GPU')
print("GPUs Available:", gpus)

model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
movenet = model.signatures['serving_default']

GPUs Available: []


### detect poes

In [2]:
def detect_pose(frame,dict_list, model=movenet, input_size=256, debug=True):

    if frame is None:
        if debug: print("detect_pose: input frame is None")
        return []

    # ensure color order: OpenCV gives BGR, MoveNet expects RGB-like ordering
    try:
        if frame.ndim == 2:  # grayscale -> convert to 3 channels
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    except Exception as e:
        if debug: print("detect_pose: color conversion failed:", e)
        return []

    # Resize to model input size
    img_resized = cv2.resize(img_rgb, (input_size, input_size))

    # MoveNet expects integer pixel values (0-255). Use int32.
    img_int = img_resized.astype(np.int32)
    input_tensor = tf.expand_dims(tf.convert_to_tensor(img_int, dtype=tf.int32), axis=0)  # shape [1,H,W,3]

    # Call model and be defensive about errors
    try:
        outputs = model(input=input_tensor)
    except Exception as e:
        if debug: print("detect_pose: model call failed:", e)
        return []

    # Look for expected output key
    if 'output_0' not in outputs:
        if debug:
            print("detect_pose: unexpected output keys from model:", list(outputs.keys()))
        return []

    keypoints_with_scores = outputs['output_0'].numpy()  # expected shape [1, 1, 17, 3]

    # guard against weird shapes
    if keypoints_with_scores.ndim != 4 or keypoints_with_scores.shape[2] != 17:
        if debug: print("detect_pose: unexpected keypoints shape")
        return []
    kps = keypoints_with_scores[0][0]
    last_kps = [row[:2] for row in kps]
    if dict_list==0 :
        return kps
    

    if debug:
        scores = kps[:,2]

    keypoints = []
    for kp in kps:
        y, x, score = float(kp[0]), float(kp[1]), float(kp[2])
        keypoints.append({'x': x, 'y': y, 'score': score})


    if dict_list==1 :
        return keypoints


def draw_keypoints(frame, keypoints, threshold=0.3):
    """Draw small circles on `frame` for keypoints with score >= threshold."""
    h, w = frame.shape[:2]
    for i, kp in enumerate(keypoints):
        if kp['score'] >= threshold:
            cx = int(kp['x'] * w)
            cy = int(kp['y'] * h)
            cv2.circle(frame, (cx, cy), 4, (0,255,0), -1)
            # optional: label index
            cv2.putText(frame, str(i), (cx+5, cy+5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 1)
    return frame


### predict standing angle

In [3]:
NOSE = 0
LEFT_SHOULDER = 5
RIGHT_SHOULDER = 6

def calculate_person_angle(keypoints_cam1, keypoints_cam2, confidence_threshold=0.3):
    try:
        required_indices = [NOSE, LEFT_SHOULDER, RIGHT_SHOULDER]
        max_index = max(required_indices)
        if (keypoints_cam1 is None or len(keypoints_cam1) <= max_index or
            keypoints_cam2 is None or len(keypoints_cam2) <= max_index):
            print("Error: Keypoint list is None, missing, or too short. A camera may have failed to detect a pose.")
            return None

        # Get keypoints from Camera 1
        ls1 = keypoints_cam1[LEFT_SHOULDER]
        rs1 = keypoints_cam1[RIGHT_SHOULDER]
        nose1 = keypoints_cam1[NOSE]

        # Get keypoints from Camera 2
        ls2 = keypoints_cam2[LEFT_SHOULDER]
        rs2 = keypoints_cam2[RIGHT_SHOULDER]
        nose2 = keypoints_cam2[NOSE]

        # Check confidence scores
        if not all(kp['score'] > confidence_threshold for kp in [ls1, rs1, nose1, ls2, rs2, nose2]):
            return "not accurate"
        x_l1, x_r1, x_n1 = ls1['x'], rs1['x'], nose1['x']
        x_l2, x_r2, x_n2 = ls2['x'], rs2['x'], nose2['x']
        dx = x_l1 - x_r1
        dz = x_l2 - x_r2
        mid_shoulder_x = (x_l1 + x_r1) / 2
        mid_shoulder_z = (x_l2 + x_r2) / 2
        nose_dir_vec_x = x_n1 - mid_shoulder_x
        nose_dir_vec_z = x_n2 - mid_shoulder_z
        p1_x, p1_z = -dz, dx
        dot_product = (p1_x * nose_dir_vec_x) + (p1_z * nose_dir_vec_z)

        if dot_product > 0:
            facing_vec_x, facing_vec_z = p1_x, p1_z
        else:
            facing_vec_x, facing_vec_z = dz, -dx
        angle_rad = math.atan2(facing_vec_x, facing_vec_z)
        angle_deg = math.degrees(angle_rad)

        return angle_deg

    except (TypeError) as e:
        print(f"Error: Invalid keypoint data format. {e}")
        return None

### cameras prepairing and use

In [4]:
def remove_black_bars_auto(frame, thresh_value=16):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, thresh_value, 255, cv2.THRESH_BINARY)
    coords = cv2.findNonZero(mask)
    if coords is None:
        return frame
    x, y, w, h = cv2.boundingRect(coords)
    return frame[y:y+h, x:x+w]

def rotate_if_needed(frame, rotate_code=None):
    if rotate_code is None:
        return frame
    return cv2.rotate(frame, rotate_code)

def resize_and_center_crop_fill(img, target_size=(640, 480)):
    target_w, target_h = target_size
    h, w = img.shape[:2]

    if (w, h) == (target_w, target_h):
        return img

    scale = max(target_w / w, target_h / h)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    x = (new_w - target_w) // 2
    y = (new_h - target_h) // 2
    cropped = resized[y:y+target_h, x:x+target_w]
    return cropped

In [5]:
cap0 = cv2.VideoCapture(0)
cap1 = cv2.VideoCapture(1)

while cap0.isOpened() and cap1.isOpened():
    ret0, frame0 = cap0.read()
    ret1, frame1 = cap1.read()
    frame1 = cv2.rotate(frame1, cv2.ROTATE_90_CLOCKWISE)
    frame1 = remove_black_bars_auto(frame1, thresh_value=16)
    frame1 = resize_and_center_crop_fill(frame1, target_size=(640, 480))

    keypoints1 = detect_pose(frame0,0)
    keypoints2 = detect_pose(frame1,0)
    kpts0 = []
    kpts1 = []

    for kp in keypoints1:
        y, x, score = float(kp[0]), float(kp[1]), float(kp[2])
        kpts0.append({'x': x, 'y': y, 'score': score})

    for kp in keypoints2:
        y, x, score = float(kp[0]), float(kp[1]), float(kp[2])
        kpts1.append({'x': x, 'y': y, 'score': score})

    frame0 = draw_keypoints(frame0, kpts0)
    frame1 = draw_keypoints(frame1, kpts1)


    degree = calculate_person_angle(kpts0, kpts1) if calculate_person_angle(kpts0, kpts1) == "not accurate" else (f"deg:{calculate_person_angle(kpts0, kpts1):.1f}")

    cv2.putText(frame0, degree, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(frame1, degree, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    combined = cv2.hconcat([frame0, frame1])

    cv2.imshow("Combined", combined)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap0.release()
cap1.release()
cv2.destroyAllWindows()
